# Course project: cleavage plane for Ca doped YBCO

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from pwscf import make_struc_doped, make_struc_undoped, write_inputs

## Z Test

Make input files (from z-1 to z-4)

In [5]:
zs = np.arange(1,5)
for z in zs:
    [struc, struc_name] = make_struc_undoped(nxy = 1, nz = z, vacuum = 40)
    infile = write_inputs(struc = struc, dirname = 'LL_test', name = struc_name, calc = 'vc-relax')

MAKING FILE
Writing LL_test/YBCO_conv_111_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_112_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_113_40vac_NOcleave_0sep_vc-relax.in
MAKING FILE
Writing LL_test/YBCO_conv_114_40vac_NOcleave_0sep_vc-relax.in


## Separation Test

In [4]:
separation = 10
nz=3

a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'Y', nxy= 2)
a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'BaO')
a=make_struc_undoped(vacuum = 20, nz = nz, separation = separation, cleave_plane = 'CuO')

## Make Doped Cell

In [6]:
a = make_struc_doped(nz = 1)

## Doped Separation Test

To make the cleaving plane on the doped sample I will need to sort all of the atoms by z height first, since they are not currently sorted within the 2xrt(2) supercell.

Maybe get atoms, sort by z, return indexes, sort chemical symbols by same indexes, then count up how many planes should be included for each cleaving.

In [48]:
nxy=1
nz = 1
alat=3.82 
blat=3.89 
clat=11.68
vacuum=0
cleave_plane='NO'
separation=0
slab = True

a = numpy.sqrt(alat**2 + blat**2)
lattice = numpy.array([[a,0,0],[0,a,0],[0,0,clat]])
symbols = ['Cu', 'Cu', 'O', 'O',
           'O','O','Ba', 'Ba',
           'Cu', 'Cu', 'O', 'O', 'O', 'O',
           'Y', 'Y',
           'O', 'O', 'O', 'O' ,'Cu', 'Cu',
           'Ba', 'Ba', 'O', 'O']
sc_pos = [[0,0,0], #Cu
          [0.5,0.5,0], #Cu
          [0.25,0.25,0], #O
          [0.75,0.75,0], #O
          [0,0,0.15918], #O
          [0.5,0.5,0.15918], #O
          [0.5, 0, 0.18061], #Ba
          [0, 0.5, 0.18061], #Ba
          [0,0,0.35332], #Cu
          [0.5,0.5,0.35332], #Cu
          [0.25,0.25,0.37835], #O
          [0.75,0.75,0.37835], #O
          [0.25,0.75,0.37935], #O
          [0.75,0.25,0.37935], #O
          [0.5,0,0.5], #Y
          [0,0.5,0.5], #Y
          [0.25,0.25,0.62065], #O
          [0.75,0.75,0.62065], #O
          [0.25,0.75,0.62165], #O
          [0.75,0.25,0.62165], #O
          [0,0,0.64668], #Cu
          [0.5,0.5,0.64668], #Cu
          [0.5, 0, 0.81939], #Ba
          [0, 0.5, 0.81939], #Ba
          [0,0,0.84082], #O
          [0.5,0.5,0.84082] #O
         ]
YBCO = Atoms(symbols=symbols, scaled_positions=sc_pos, cell=lattice)

#make an x/y supercell of 2 and dope 1/8 Y --> Ca
multiplier = numpy.identity(3)
multiplier[0,0]=2
multiplier[1,1]=2
supercell = make_supercell(YBCO, multiplier)

temp_sym = supercell.get_chemical_symbols()
temp_sym[15] = 'Ca'
supercell.set_chemical_symbols(temp_sym)
    

#make the supercell of the 2rt(2) doped supercell in the z direction
multiplier = numpy.identity(3)
multiplier[2,2]=nz
supercell = make_supercell(supercell, multiplier)

#sort the atoms so cleave plane can easily be determined
pos = supercell.get_positions()
sym = supercell.get_chemical_symbols()
args = numpy.argsort(pos[:,2])
supercell.set_positions(pos[args])
supercell.set_chemical_symbols([sym[j] for j in args])


if slab:
    #make the position of the 'capping' layer
    Cu_layer = Atoms(symbols = ['Cu', 'Cu', 'O', 'O'], 
                     scaled_positions = [[0.0,0.0, 0.0],
                                  [0.5,0.5,0],
                                  [0.25,0.25,0], 
                                  [0.75,0.75,0]],
                   cell = numpy.array([[a,0,0],[0,a,0],[0,0,clat]]))
    #make a supercell of the capping layer
    multiplier = numpy.identity(3)
    multiplier[0,0]=2
    multiplier[1,1]=2
    Cu_layer = make_supercell(Cu_layer, multiplier)

    #cap the unit cell
    supercell = stack(supercell, Cu_layer)

    #add vacuum
    add_vacuum(supercell, vacuum)

#find the plane to cleave on (closest to the middle)
    if cleave_plane == 'CuO':
        split = int(nz/2)*104 + 16
        
    elif cleave_plane == "BaO":
        split = int(nz/2)*104 + 32
        
    elif cleave_plane == "Y":
        split = int(nz/2)*104 + 64
    
    if cleave_plane != "NO":
        temp_pos = supercell.get_positions()
        temp_pos[split:,2] += separation #add separation in z to all atoms after cleave plane
        supercell.set_positions(temp_pos)
        temp_cell = supercell.get_cell()
        temp_cell[2][2] += separation #add separation to cell height so vacuum is unchanged
        supercell.set_cell(temp_cell)
        
#make the supercell of the 2rt(2) doped supercell in x/y
multiplier = numpy.identity(3)
multiplier[0,0]=nxy
multiplier[1,1]=nxy
supercell = make_supercell(supercell, multiplier)

#output to cif
name = f'YBCO_rt2_{nxy}{nxy}{nz}_{vacuum}vac_{cleave_plane}cleave_{separation}sep'
write(f'{name}.cif', supercell)

# Inital relax calculation (RUNS A CALC)

In [ ]:
def relax_YBCO(nk, ecut):
    pseudopots = {'Y': PseudoPotential(ptype='uspp', element='Y', functional='LDA', name='Y.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Ba': PseudoPotential(ptype='uspp', element='Ba', functional='LDA', name='Ba.pz-spn-rrkjus_psl.1.0.0.UPF'),
                  'Cu': PseudoPotential(ptype='uspp', element='Cu', functional='LDA', name='Cu.pz-d-rrkjus.UPF'),
                  'O': PseudoPotential(ptype='uspp', element='O', functional='LDA', name='O.pz-rrkjus.UPF')}
    struc = make_struc_non_doped()
    kpts = Kpoints(gridsize=[nk, nk, nk], option='automatic', offset=True)
    dirname = 'YBCO_relax_ecut_{}_nk_{}'.format(ecut, nk)
    runpath = Dir(path=os.path.join(os.environ['WORKDIR'], "Project", dirname))
    input_params = PWscf_inparam({
        'CONTROL': {
            'calculation': 'relax',
            'pseudo_dir': os.environ['QE_POTENTIALS'],
            'outdir': runpath.path,
            'tstress': True,
            'tprnfor': True,
            'disk_io': 'none'
        },
        'SYSTEM': {
            'ecutwfc': ecut,
            'ecutrho': ecut * 12,
            'occupations': 'smearing',
            'smearing': 'mp',
            'degauss': 0.02
             },
        'ELECTRONS': {
            'diagonalization': 'david',
            'mixing_beta': 0.7,
            'conv_thr': 1e-7,
        },
        'IONS': {
            'ion_dynamics': 'bfgs'
        },
        'CELL': {},
        })

    output_file = run_qe_pwscf(runpath=runpath, struc=struc,  pseudopots=pseudopots,
                               params=input_params, kpoints=kpts, ncpu=16)
    output = parse_qe_pwscf_output(outfile=output_file)
    return output

def kpts_scan():
    ecut = 30
    nk = np.arange(1, 2)
    energy=[]
    for i in nk:
        output = relax_YBCO(ecut=ecut, nk=i)
        energy.append(output['energy'])
        print(i)
    return energy